In [1]:
# Importing necessary libraries
library("tidyverse")
library("ggplot2")
library("ggrepel")
library("ggcorrplot")
library("DT")
library(dplyr)
library(tidyr)

install.packages("dplyr")

# Load the dplyr package
library(dplyr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error in library("ggrepel"): there is no package called ‘ggrepel’


# COVID-19_cases_plus_census

In [2]:
# Retrieve "COVID-19_cases_plus_census.csv" file
dataset_plus_census <- read.csv("https://raw.githubusercontent.com/yamphurai/Data_Mining_Project1/refs/heads/main/COVID-19_cases_plus_census.csv")

## Freature Processing

In [3]:
# Create a new column 'rent_under_50_percent' to add the values from columns with rent 50% and below
dataset_plus_census <- dataset_plus_census %>%
  mutate(rent_under_50_percent = rent_40_to_50_percent +
                                  rent_35_to_40_percent +
                                  rent_30_to_35_percent +
                                  rent_25_to_30_percent +
                                  rent_20_to_25_percent +
                                  rent_15_to_20_percent +
                                  rent_10_to_15_percent +
                                  rent_under_10_percent)

In [6]:
length(names(dataset_plus_census))

[1] 260

In [7]:
# Checking the new values in the new column
str(dataset_plus_census$rent_under_50_percent)

 num [1:3142] 411 16148 13483 8510 4211 ...


The sum of the values of the first row and columns with rents <50% is $411. The sum for the second row is $16148. This indicates the the new column has been created correctly.

In [8]:
# For the analysis on COVID-19 cases, there could be minimum correlation between "race" and impact. Also, the analysis will already
# including the living conditions of the public. Thus, the feature columns regarding race can be dropped.
dataset_plus_census <- dataset_plus_census %>%
  select(-rent_40_to_50_percent, -rent_35_to_40_percent, -rent_30_to_35_percent, -rent_25_to_30_percent, -rent_burden_not_computed, -rent_over_50_percent,
         -rent_20_to_25_percent, -rent_15_to_20_percent, -rent_10_to_15_percent, -rent_under_10_percent)

In [9]:
length(names(dataset_plus_census))

[1] 250

In [10]:
# We will also not consider structure of the families for our analysis and rather maintain focus on the financial structure of the families
dataset_plus_census <- dataset_plus_census %>%
  select(-families_with_young_children, -two_parent_families_with_young_children, -two_parents_in_labor_force_families_with_young_children, -two_parents_father_in_labor_force_families_with_young_children,
         -two_parents_mother_in_labor_force_families_with_young_children, -two_parents_not_in_labor_force_families_with_young_children, -one_parent_families_with_young_children, -father_one_parent_families_with_young_children,
        -father_in_labor_force_one_parent_families_with_young_children)

In [11]:
length(names(dataset_plus_census))

[1] 241

In [12]:
# Similar to the above reasoning, we will ignore statistics on housing data
dataset_plus_census <- dataset_plus_census %>%
  select(-housing_units, -vacant_housing_units, -vacant_housing_units_for_rent, -vacant_housing_units_for_sale, -owner_occupied_housing_units,
         -million_dollar_housing_units, -mortgaged_housing_units, -owner_occupied_housing_units_lower_value_quartile, -owner_occupied_housing_units_median_value,
        -owner_occupied_housing_units_upper_value_quartile, -occupied_housing_units, -married_households, -housing_units_renter_occupied)

In [13]:
length(names(dataset_plus_census))

[1] 228

In [14]:
# Create a new columns 'income_less_50K', "income_50K_100K', "income_100K-150K" , "income_150K_more" to group the columns related to income levels
dataset_plus_census <- dataset_plus_census %>%
  mutate(income_less_50K = income_less_10000 + income_10000_14999 + income_15000_19999 + income_20000_24999 + income_25000_29999 + income_30000_34999 +
         income_35000_39999  + income_40000_44999 + income_45000_49999)

dataset_plus_census <- dataset_plus_census %>%
  mutate(income_50K_100K = income_50000_59999 + income_60000_74999 + income_75000_99999)

dataset_plus_census <- dataset_plus_census %>%
  mutate(income_100K_150K = income_100000_124999 + income_125000_149999)

dataset_plus_census <- dataset_plus_census %>%
  mutate(income_150K_more = income_150000_199999 + income_200000_or_more)


In [15]:
# Now the old columns related to the income levels can be dropped
dataset_plus_census <- dataset_plus_census %>%
  select(-income_less_10000, -income_10000_14999, -income_15000_19999, -income_20000_24999, -income_25000_29999, -income_30000_34999, -income_35000_39999,
         -income_40000_44999, -income_45000_49999, -income_50000_59999, -income_60000_74999, -income_75000_99999, -income_100000_124999, -income_125000_149999,
        -income_150000_199999, -income_200000_or_more)

In [16]:
length(names(dataset_plus_census))

[1] 216

In [17]:
# Data on the dwelling and the built year of the houses are not significant to our analysis so those columns are dropped
dataset_plus_census <- dataset_plus_census %>%
  select(-renter_occupied_housing_units_paying_cash_median_gross_rent, -dwellings_1_units_detached, -dwellings_1_units_attached, -dwellings_2_units, -dwellings_3_to_4_units,
         -dwellings_5_to_9_units, -dwellings_10_to_19_units, -dwellings_20_to_49_units, -dwellings_50_or_more_units, -mobile_homes, -housing_built_2005_or_later,
         -housing_built_2000_to_2004, -housing_built_1939_or_earlier)

In [18]:
length(names(dataset_plus_census))

[1] 203

In [19]:
# We will group the ages of males and females from 0-20, 21-49, 50 above. So the relevant feature columns are dropped
dataset_plus_census <- dataset_plus_census %>%
  mutate(male_0_20 = male_under_5 + male_5_to_9 + male_10_to_14 + male_15_to_17 + male_18_to_19 + male_20)

dataset_plus_census <- dataset_plus_census %>%
  mutate(male_21_49 = male_21 + male_22_to_24 + male_25_to_29 + male_30_to_34 + male_35_to_39 + male_40_to_44 + male_45_to_49)

dataset_plus_census <- dataset_plus_census %>%
  mutate(male_50_above = male_50_to_54 + male_55_to_59 + male_60_61 + male_62_64 + male_65_to_66 + male_67_to_69 + male_67_to_69 + male_70_to_74 + male_75_to_79 + male_80_to_84 + male_85_and_over)

dataset_plus_census <- dataset_plus_census %>%
  mutate(female_0_20 = female_under_5 + female_5_to_9 + female_10_to_14 + female_15_to_17 + female_18_to_19 + female_20)

dataset_plus_census <- dataset_plus_census %>%
  mutate(female_21_49 = female_21 + female_22_to_24 + female_25_to_29 + female_30_to_34 + female_35_to_39 + female_40_to_44 + female_45_to_49)

dataset_plus_census <- dataset_plus_census %>%
  mutate(female_50_above = female_50_to_54 + female_55_to_59 + female_60_to_61 + female_62_to_64 + female_65_to_66 + female_67_to_69 + female_70_to_74 + female_75_to_79 + female_75_to_79 + female_80_to_84 + female_85_and_over)

# Drop the feature columns now
dataset_plus_census <- dataset_plus_census %>%
  select(-male_under_5, -male_5_to_9, -male_10_to_14, -male_15_to_17, -male_18_to_19, -male_20, -male_21, -male_22_to_24, -male_25_to_29, -male_30_to_34,
         -male_35_to_39, -male_40_to_44, -male_45_to_49, -male_50_to_54, -male_55_to_59, -male_60_61, -male_62_64, -male_65_to_66, -male_67_to_69, -male_67_to_69,
         -male_70_to_74, -male_75_to_79, -male_80_to_84, -male_85_and_over,

         -female_under_5, -female_5_to_9, -female_10_to_14, -female_15_to_17, -female_18_to_19, -female_20, -female_21, -female_22_to_24, -female_25_to_29,
         -female_30_to_34, -female_35_to_39, -female_40_to_44, -female_45_to_49, -female_50_to_54, -female_55_to_59, -female_60_to_61, -female_62_to_64, -female_65_to_66, -female_67_to_69, -female_70_to_74,
         -female_75_to_79, -female_80_to_84, -female_85_and_over)

In [20]:
length(names(dataset_plus_census))

[1] 163

In [21]:
# Drop the columns with values related to race
dataset_plus_census <- dataset_plus_census %>%
  select(-white_including_hispanic, -black_including_hispanic, -amerindian_including_hispanic, -asian_including_hispanic)

In [22]:
length(names(dataset_plus_census))

[1] 159

In [23]:
# The data on the education status of the people are not significant to our analysis so the relevant columns are dropped
dataset_plus_census <- dataset_plus_census %>%
  select(-one_year_more_college, -masters_degree, -less_one_year_college, -high_school_diploma, -bachelors_degree, -associates_degree)

In [24]:
length(names(dataset_plus_census))

[1] 153

In [25]:
# Since we are already considering the age of males and females separately, we can drop other relevant columns
dataset_plus_census <- dataset_plus_census %>%
  select(-population_1_year_and_over, -population_3_years_over, -pop_determined_poverty_status, -pop_25_64)


In [26]:
length(names(dataset_plus_census))

[1] 149

In [27]:
# Group all the data of the columns containing data related to commuting to work regardless of time spend to commute
dataset_plus_census <- dataset_plus_census %>%
  mutate(commute = commuters_16_over + commute_less_10_mins + commute_60_more_mins + commute_35_44_mins + commuters_drove_alone + commuters_by_subway_or_elevated +
         commuters_by_car_truck_van + commuters_by_carpool + commuters_by_bus + commute_90_more_mins + commute_60_89_mins + commute_40_44_mins + commute_35_39_mins +
         commute_5_9_mins + commuters_by_public_transportation)

In [28]:
# Dropping the relevant columns for commute time
dataset_plus_census <- dataset_plus_census %>%
  select(-commute_10_14_mins, -commute_15_19_mins, -commute_20_24_mins, -commute_25_29_mins, -commute_30_34_mins, -commute_45_59_mins, -commute_5_9_mins, -commute_35_39_mins, -commute_40_44_mins,
-commute_60_89_mins,-commute_90_more_mins, -commuters_by_bus, -commuters_by_car_truck_van, -commuters_by_carpool, -commuters_by_subway_or_elevated, -commuters_drove_alone, -commute_60_more_mins,
-commute_35_44_mins, -commute_60_more_mins, -commute_less_10_mins, -commuters_16_over)

In [29]:
length(names(dataset_plus_census))

[1] 130

In [30]:
# We are already considering male/female and age redated data. So these columns are dropped
dataset_plus_census <- dataset_plus_census %>%
  select(-white_male_55_64, -white_male_45_54, -hispanic_male_55_64, -hispanic_male_45_54, -black_male_55_64, -black_male_45_54, -asian_male_55_64, -asian_male_45_54)

In [31]:
length(names(dataset_plus_census))

[1] 122

In [32]:
# Drop irrelevant columns
dataset_plus_census <- dataset_plus_census %>%
  select(-households, -income_per_capita, -percent_income_spent_on_rent, -median_rent, -hispanic_any_race, -speak_only_english_at_home,
        -speak_spanish_at_home, -speak_spanish_at_home_low_english, -pop_5_years_over, -pop_15_and_over, -pop_never_married, -pop_now_married, -pop_separated, -pop_widowed,
         -pop_divorced, -pop_25_years_over, -poverty, -workers_16_and_over, -some_college_and_associates_degree, -sales_office_employed, -one_car, -no_car, -two_cars, -three_cars, -four_more_cars,
          -aggregate_travel_time_to_work,
         -occupation_management_arts, -occupation_natural_resources_construction_maintenance, -occupation_production_transportation_material, -occupation_sales_office, -occupation_services, -not_us_citizen_pop,
         -management_business_sci_arts_employed, -male_male_households, -male_45_to_64, -male_45_64_some_college, -male_45_64_high_school, -male_45_64_grade_9_12, -male_45_64_less_than_9_grade,
         -male_45_64_graduate_degree, -male_45_64_associates_degree, -less_than_high_school_graduate, -in_undergrad_college, -in_school, -in_grades_9_to_12, -in_grades_5_to_8, -in_grades_1_to_4,
         -high_school_including_ged, -group_quarters, -graduate_professional_degree, -gini_index, -female_female_households, -employed_wholesale_trade, -employed_transportation_warehousing_utilities,
         -employed_science_management_admin_waste, -employed_retail_trade, -employed_public_administration, -employed_other_services_not_public_admin, -employed_manufacturing, -employed_information,
         -employed_finance_insurance_real_estate, -employed_education_health_social, -employed_construction, -employed_arts_entertainment_recreation_accommodation_food, -employed_agriculture_forestry_fishing_hunting_mining,
         -households_public_asst_or_food_stamps, -different_house_year_ago_same_city, -different_house_year_ago_different_city, -children_in_single_female_hh, -children, -bachelors_degree_or_higher_25_64,
         -bachelors_degree_2, -pop_in_labor_force, -pop_16_over, -not_in_labor_force, -civilian_labor_force, -armed_forces, -households_retirement_income, -commuters_by_public_transportation, -male_45_64_bachelors_degree)





In [33]:
dataset_plus_census <- dataset_plus_census %>%
  select(-county_fips_code, -nonfamily_households, -family_households, -median_year_structure_built, -two_or_more_races_pop, -not_hispanic_pop, -no_cars, -rent_under_50_percent, -do_date)

In [34]:
# Remaining feature columns
print(length(names(dataset_plus_census)))
print(names(dataset_plus_census))

[1] 33
 [1] "county_name"      "state"            "state_fips_code"  "date"            
 [5] "confirmed_cases"  "deaths"           "geo_id"           "total_pop"       
 [9] "male_pop"         "female_pop"       "median_age"       "white_pop"       
[13] "black_pop"        "asian_pop"        "hispanic_pop"     "amerindian_pop"  
[17] "other_race_pop"   "median_income"    "employed_pop"     "unemployed_pop"  
[21] "walked_to_work"   "worked_at_home"   "income_less_50K"  "income_50K_100K" 
[25] "income_100K_150K" "income_150K_more" "male_0_20"        "male_21_49"      
[29] "male_50_above"    "female_0_20"      "female_21_49"     "female_50_above" 
[33] "commute"         


In [35]:
# Rearrange the columns in the specified order
dataset_plus_census <- dataset_plus_census %>%
  select(
    date, state, state_fips_code, county_name, geo_id, median_age, male_pop, female_pop, total_pop,
    white_pop, black_pop, asian_pop, hispanic_pop, amerindian_pop, other_race_pop, median_income,
    income_less_50K, income_50K_100K, income_100K_150K, income_150K_more, employed_pop,
    unemployed_pop, walked_to_work, commute, male_0_20, male_21_49, male_50_above,
    female_0_20, female_21_49, female_50_above
  )

# Save the updated dataset to a CSV file
write.csv(dataset_plus_census, "updated_dataset_plus_census.csv", row.names = FALSE)

In [37]:
print(length(names(dataset_plus_census)))
print(names(dataset_plus_census))

[1] 30
 [1] "date"             "state"            "state_fips_code"  "county_name"     
 [5] "geo_id"           "median_age"       "male_pop"         "female_pop"      
 [9] "total_pop"        "white_pop"        "black_pop"        "asian_pop"       
[13] "hispanic_pop"     "amerindian_pop"   "other_race_pop"   "median_income"   
[17] "income_less_50K"  "income_50K_100K"  "income_100K_150K" "income_150K_more"
[21] "employed_pop"     "unemployed_pop"   "walked_to_work"   "commute"         
[25] "male_0_20"        "male_21_49"       "male_50_above"    "female_0_20"     
[29] "female_21_49"     "female_50_above" 
